# Model Training
Welcome to the 'Model Training and Prediction' notebook, a crucial facet of our project's data science pipeline. In this notebook, we offer a meticulous examination of our rigorous model development process. The pipeline starts by accepting training data, followed by fitting three distinct types of models to it: Random Forest, Gradient Boosted Tree, and XGBoost. The initial stages include encoding categorical variables and executing Recursive Feature Elimination (RFE) for feature selection. This is succeeded by the application of genetic algorithms to hyperparameter tuning, operating in tandem with a cross-validation routine. Subsequently, the best model is selected based on the highest F1 score, indicating the balance between precision and recall. Finally, the selected model is utilized to predict the outcomes for the current week's round of NRL matches. This process is iterative and cyclical, with the potential for revisiting earlier stages based on the model's performance. Let us proceed with this in-depth exploration.

## Set up Environment
This code segment is setting up the environment for the model training pipeline. It begins by importing sys and pathlib - Python libraries used for managing system parameters and file paths, respectively.

The code then updates the system path to include the "functions" directory. This allows for the import of custom modules `modelling_functions`, `model_properties`, and `training_config` which are stored in this directory. These modules contain custom functions and configuration settings that are critical for the later stages of data preprocessing, model training, and prediction.

Following this, the `project_root` variable is defined. This is achieved by using the pathlib library to establish the root directory of the project.

Finally, the `db_path` variable is constructed. This is the relative path to the SQLite database "footy-tipper-db.sqlite", which is located in the "data" directory of the project root. This path will be used for database connectivity throughout the pipeline.

In [1]:
# import libraries
import os
import sys
import pathlib

cwd = os.getcwd()

# get the parent directory
parent_dir = os.path.dirname(cwd)

# add the parent directory to the system path
sys.path.insert(0, parent_dir)

# Get to the root directory
project_root = pathlib.Path().absolute().parent

# import functions from common like this:
from pipeline.common.model_training import (
    training_config as tc,
    modelling_functions as mf,
    model_properties as mp
)

from pipeline.common.model_prediciton import prediction_functions as pf

## Get data
Our process starts by establishing the root directory of the project and constructing the relative path to the 'footy-tipper-db.sqlite' database located within the 'data' directory. We then connect to this SQLite database and use a SQL query housed in the 'footy_tipping_data.sql' file, found in the 'sql' directory, to extract the required data. This data is loaded into a pandas DataFrame, footy_tipping_data, serving as the basis for our subsequent modeling activities. Upon successful extraction of the data, we ensure the database connection is closed, maintaining good coding practice and resource management.

In [2]:
training_data = mf.get_training_data(
    db_path = project_root / "data" / "footy-tipper-db.sqlite", 
    sql_file = project_root / 'pipeline/common/sql/training_data.sql')

training_data

,game_id,round_id,round_name,game_number,game_state_name,start_time,start_time_utc,venue_name,city,crowd,...,away_prev_result_diff,prev_result_diff,home_elo,away_elo,elo_diff,home_elo_prob,away_elo_prob,elo_draw_prob,elo_prob_diff,home_ground_advantage
0,2.018111e+10,1.0,Round 1,1.0,Final,1.520540e+09,1.520500e+09,Netstrata Jubilee Stadium,Sydney,14457.0,...,0.0,0.0,1498.870784,1510.433285,-11.562501,0.407142,0.428924,0.163934,-0.021782,1.595341
1,2.018111e+10,1.0,Round 1,2.0,Final,1.520618e+09,1.520579e+09,McDonald Jones Stadium,Newcastle,23516.0,...,0.0,0.0,1484.773765,1496.165275,-11.391510,0.407348,0.428718,0.163934,-0.021370,7.618695
2,2.018111e+10,1.0,Round 1,3.0,Final,1.520622e+09,1.520586e+09,1300SMILES Stadium,Townsville,15900.0,...,0.0,0.0,1506.201823,1503.250474,2.951349,0.424602,0.411464,0.163934,0.013138,0.903241
3,2.018111e+10,1.0,Round 1,4.0,Final,1.520699e+09,1.520660e+09,Accor Stadium,Sydney,18243.0,...,0.0,0.0,1493.852845,1500.998071,-7.145227,0.412455,0.423611,0.163934,-0.011156,-6.158996
4,2.018111e+10,1.0,Round 1,5.0,Final,1.520698e+09,1.520669e+09,Other,Perth,38824.0,...,0.0,0.0,1490.491710,1483.482050,7.009659,0.429483,0.406583,0.163934,0.022900,2.016667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181,2.023111e+10,29.0,Finals Week 2,1.0,Final,1.694807e+09,1.694771e+09,AAMI Park,Melbourne,19534.0,...,-1.0,22.0,1536.879637,1514.591676,22.287961,0.516831,0.448081,0.035088,0.068750,12.025141
1182,2.023111e+10,29.0,Finals Week 2,2.0,Final,1.694888e+09,1.694844e+09,Go Media Stadium,Auckland,26083.0,...,-20.0,32.0,1506.199870,1539.709974,-33.510104,0.447021,0.534357,0.018622,-0.087335,0.242179
1183,2.023111e+10,30.0,Finals Week 3,1.0,Final,1.695412e+09,1.695376e+09,Accor Stadium,Sydney,35578.0,...,26.0,0.0,1565.935167,1536.573385,29.361782,0.526589,0.438324,0.035088,0.088265,8.917399
1184,2.023111e+10,30.0,Finals Week 3,2.0,Final,1.695499e+09,1.695463e+09,Suncorp Stadium,Brisbane,52273.0,...,26.0,0.0,1553.849515,1522.439615,31.409900,0.529407,0.435505,0.035088,0.093902,5.694470


In [3]:
training_data["punt_odds_diff"]

0       0.155035
1       0.000000
2       0.167768
3      -0.595143
4       0.333050
          ...   
1181    0.402057
1182    0.220276
1183    0.633333
1184    0.549748
1185    0.208333
Name: punt_odds_diff, Length: 1186, dtype: float64

## Modelling
During the modelling phase, the `train_and_select_best_model` function, part of our `modelling_functions` module, is invoked. This function initiates the training of three distinct models: XGBoost, Random Forest, and Gradient Boosting Classifier. It takes as input the footy tipping data, predictor variables, the outcome variable, and several configuration settings like whether to use Recursive Feature Elimination (RFE), the number of cross-validation folds, and the optimization metric, all sourced from the `training_config` module.

The function first identifies categorical columns in the feature set for one-hot encoding, creating dummy variables for categorical features. Depending on the choice of using RFE, a feature elimination step may be included in the pipeline. Each model subsequently undergoes hyperparameter tuning using a genetic algorithm, facilitated by the `GASearchCV` function.

All the models are then trained and evaluated through cross-validation. The best model, or `footy_tipper`, is selected based on the superior performance on the chosen optimization metric. Additionally, a `LabelEncoder`(`label_encoder`), used to encode the categorical target variable, is returned. This LabelEncoder is specific to the model that performed best. The selected model, encapsulated in a pipeline with pre-processing steps and hyperparameter tuning, is now ready for the prediction phase.

In [4]:
footy_tipper, label_encoder = mf.train_and_select_best_model(
    training_data, tc.predictors, tc.outcome_var,
    tc.use_rfe, tc.num_folds, tc.opt_metric
)
footy_tipper


Model training: XGBClassifier
gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	100   	0.677421	0.00833221 	0.69483    	0.654363   
1  	128   	0.685212	0.00630662 	0.696497   	0.669553   
2  	132   	0.689352	0.00490293 	0.696529   	0.673758   
3  	135   	0.69231 	0.00395287 	0.697366   	0.677123   
4  	133   	0.694223	0.00283518 	0.697366   	0.680506   
5  	134   	0.695568	0.00203433 	0.699894   	0.683878   
6  	136   	0.696543	0.0017771  	0.699894   	0.68809    
7  	132   	0.697331	0.00134531 	0.699894   	0.694827   
8  	133   	0.697629	0.00183043 	0.699894   	0.68976    
9  	146   	0.6984  	0.00144614 	0.699894   	0.690614   
10 	130   	0.698687	0.00187508 	0.702397   	0.688934   
11 	128   	0.699446	0.00156408 	0.705776   	0.690614   
12 	144   	0.699975	0.00243148 	0.705776   	0.690604   
13 	139   	0.701119	0.00287374 	0.705776   	0.691448   
14 	148   	0.702262	0.00280204 	0.705776   	0.69566    
15 	136   	0.703943	0.00244018 	0.705776   	0.699043   
16 	147   	0.7047

Pipeline(steps=[('one_hot_encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['round_name', 'venue_name',
                                                   'city', 'team_home',
                                                   'team_away',
                                                   'broadcast_channel1',
                                                   'broadcast_channel2',
                                                   'broadcast_channel3',
                                                   'game_day'])])),
                ('feature_elimination',
                 RFECV(cv=5,
                       estimator=XGBClassifier(base_score=Non...
                                        'learning_rate': <sklearn_genetic.space.space.Continuous object at 0x14cd431d0>,
                                        'max_depth': <sklearn_genetic.space.space.Integer object at 0x14cd438d0>,
                                        'n_estimators': <sklearn_genetic.space.space.Integer object at 0x1054b4d90>,
                                        'subsample': <sklearn_genetic.space.space.Continuous object at 0x14cd43950>},
                            population_size=100, return_train_score=True,
                            scoring='accuracy'))])

### Display feature importance
The `get_feature_importance` function retrieves feature importances from a trained scikit-learn pipeline. It accounts for different transformations, such as one-hot encoding and recursive feature elimination. The function then returns a sorted DataFrame listing each feature alongside its respective importance, aiding in understanding the model's decision-making process.

In [5]:
# feature_importance_df = mp.get_feature_importances_from_pipeline(footy_tipper, tc.predictors)
# feature_importance_df

## Save Model
The `save_models` function stores the trained LabelEncoder and Pipeline objects to the disk. This allows for easy retrieval and reuse in future model prediction tasks, without the need to retrain these components. The objects are stored in a designated 'models' directory under the project root path, ensuring organized and consistent storage.

In [6]:
mf.save_models(label_encoder, footy_tipper, project_root)

LabelEncoder saved to models/label_encoder.pkl
Pipeline saved to models/footy_tipper.pkl


## Predict
The final stage of the pipeline involves predicting the outcomes of the current week's NRL matches. This is achieved by connecting to the SQLite database and extracting the required data. The trained model and LabelEncoder are then loaded from the disk, and the prediction is performed using the `model_predictions` function. The predictions are stored in the 'predictions' table of the database, allowing for easy retrieval and analysis.

In [7]:
label_encoder, footy_tipper = pf.load_models(project_root)

Loading models...
Label encoder loaded
Model pipeline loaded


In [8]:
inference_data = pf.get_inference_data(
    db_path = project_root / "data" / "footy-tipper-db.sqlite", 
    sql_file = project_root / 'pipeline/common/sql/inference_data.sql')
inference_data

Getting inference data...


,game_id,round_id,round_name,game_number,game_state_name,start_time,start_time_utc,venue_name,city,crowd,...,away_prev_result_diff,prev_result_diff,home_elo,away_elo,elo_diff,home_elo_prob,away_elo_prob,elo_draw_prob,elo_prob_diff,home_ground_advantage
0,2.024111e+10,1.0,Round 1,1.0,Pre Game,1.709404e+09,1.709433e+09,Other,Las Vegas,None,...,0.0,0.0,1503.793726,1495.207178,8.586547,0.431378,0.404687,0.163934,0.026691,-2.687567
1,2.024111e+10,1.0,Round 1,2.0,Pre Game,1.709411e+09,1.709440e+09,Other,Las Vegas,None,...,0.0,0.0,1507.448964,1532.179636,-24.730673,0.459348,0.522031,0.018622,-0.062683,0.176200
2,2.024111e+10,1.0,Round 1,3.0,Pre Game,1.709842e+09,1.709802e+09,McDonald Jones Stadium,Newcastle,None,...,0.0,0.0,1511.735115,1488.497205,23.237910,0.518143,0.446769,0.035088,0.071374,7.870833
3,2.024111e+10,1.0,Round 1,4.0,Pre Game,1.709928e+09,1.709881e+09,Go Media Stadium,Auckland,None,...,0.0,0.0,1506.360356,1509.448144,-3.087788,0.417336,0.418729,0.163934,-0.001393,9.613233
4,2.024111e+10,1.0,Round 1,5.0,Pre Game,1.709928e+09,1.709889e+09,AAMI Park,Melbourne,None,...,0.0,0.0,1512.449999,1538.408850,-25.958851,0.457620,0.523758,0.018622,-0.066137,2.547533
5,2.024111e+10,1.0,Round 1,6.0,Pre Game,1.710005e+09,1.709966e+09,CommBank Stadium,Sydney,None,...,0.0,0.0,1497.199055,1465.957559,31.241496,0.529176,0.435737,0.035088,0.093439,12.672667
6,2.024111e+10,1.0,Round 1,7.0,Pre Game,1.710009e+09,1.709973e+09,Cbus Super Stadium,Gold Coast,None,...,0.0,0.0,1485.165233,1483.119547,2.045686,0.423512,0.412553,0.163934,0.010959,4.675700
7,2.024111e+10,1.0,Round 1,8.0,Pre Game,1.710083e+09,1.710047e+09,Suncorp Stadium,Brisbane,None,...,0.0,0.0,1490.023865,1505.036684,-15.012819,0.402995,0.433071,0.163934,-0.030076,1.246333


In [9]:
inference_data["punt_odds_diff"]

0   -0.095238
1   -0.222005
2    0.355793
3    0.197650
4   -0.123119
5    0.309902
6    0.292353
7   -0.327336
Name: punt_odds_diff, dtype: float64

In [10]:
predictions_df = pf.model_predictions(footy_tipper, inference_data, label_encoder)
predictions_df

Making predictions...


,game_id,home_team_result,home_team_win_prob,home_team_lose_prob
0,2.024111e+10,Win,0.594087,0.405913
1,2.024111e+10,Win,0.517144,0.482856
2,2.024111e+10,Win,0.661568,0.338432
3,2.024111e+10,Win,0.680153,0.319847
4,2.024111e+10,Win,0.628640,0.371360
5,2.024111e+10,Win,0.547100,0.452900
6,2.024111e+10,Win,0.675099,0.324901
7,2.024111e+10,Win,0.550196,0.449804


In [11]:
pf.save_predictions_to_db(
    predictions_df, 
    project_root / "data" / "footy-tipper-db.sqlite", 
    project_root / 'pipeline/common/sql/create_table.sql', 
    project_root / 'pipeline/common/sql/insert_into_table.sql'
)

Saving predictions to database...


# this is the sending bit

In [12]:
from dotenv import load_dotenv
from pipeline.common.model_prediciton import prediction_functions as pf
from pipeline.common.use_predictions import sending_functions as sf

# Now construct the relative path to your SQLite database
db_path = project_root / "data" / "footy-tipper-db.sqlite"
secrets_path = project_root / "secrets.env"
json_path = project_root / "service-account-token.json"

load_dotenv(dotenv_path=secrets_path)

True

In [18]:
import sqlite3
import pandas as pd
# Connect to the SQLite database
con = sqlite3.connect(str(db_path))

# Read SQL query from external SQL file
with open(project_root / 'pipeline/common' / 'sql/prediction_table.sql', 'r') as file:
    query = file.read()

# Execute the query and fetch the results into a data frame
predictions = pd.read_sql_query(query, con)

# Disconnect from the SQLite database
con.close()

predictions

,game_id,home_team_result,team_home,position_home,team_head_to_head_odds_home,team_away,position_away,team_head_to_head_odds_away,home_team_win_prob,home_team_lose_prob,round_id,competition_year,round_name
0,20241110110,Win,Manly-Warringah Sea Eagles,0,2.10,South Sydney Rabbitohs,0,1.75,0.594087,0.405913,1,2024,Round 1
1,20241110120,Win,Sydney Roosters,0,2.41,Brisbane Broncos,0,1.57,0.517144,0.482856,1,2024,Round 1
2,20241110130,Win,Newcastle Knights,0,1.42,Canberra Raiders,0,2.87,0.661568,0.338432,1,2024,Round 1
3,20241110140,Win,New Zealand Warriors,0,1.60,Cronulla-Sutherland Sharks,0,2.34,0.680153,0.319847,1,2024,Round 1
4,20241110150,Win,Melbourne Storm,0,2.15,Penrith Panthers,0,1.70,0.628640,0.371360,1,2024,Round 1
5,20241110160,Win,Parramatta Eels,0,1.47,Canterbury-Bankstown Bulldogs,0,2.70,0.547100,0.452900,1,2024,Round 1
6,20241110170,Win,Gold Coast Titans,0,1.49,St. George Illawarra Dragons,0,2.64,0.675099,0.324901,1,2024,Round 1
7,20241110180,Win,Dolphins,0,2.76,North Queensland Cowboys,0,1.45,0.550196,0.449804,1,2024,Round 1


In [20]:
tipper_picks = sf.get_tipper_picks(predictions)
tipper_picks

,team,price,price_min
0,Manly-Warringah Sea Eagles,2.10,1.683254
1,Sydney Roosters,2.41,1.933697
3,New Zealand Warriors,1.60,1.470257
4,Melbourne Storm,2.15,1.590736
7,Dolphins,2.76,1.817533


In [23]:
sf.upload_df_to_drive(
    predictions, 
    json_path, 
    os.getenv('FOLDER_ID'), 
    "predictions.csv"
)

File ID: 1ovpAGAmsUj45j3CWS30ENmxjp3mRpedu


In [29]:
reg_reagan = sf.generate_reg_regan_email(
    predictions, 
    tipper_picks, 
    os.getenv('OPENAI_KEY'), 
    os.getenv('FOLDER_URL'),
    1
)

print(reg_reagan)

Subject: Round 1 Rundown and Value Picks from The Footy Tipper

G'day mates,

Hope you are ready for some top-notch footy action because Round 1 is knocking at the door. Been running the old brain cells through the new-and-improved Footy Tipper to give you lot the best possible tips.

Let's dive into these games, eh?

First off, the stinking Manly-Warringah Sea Eagles are playing against the South Sydney Rabbitohs. As much as it pains me to say it, those Manly fellas have been tipped to win, but hey the Footy Tipper isn’t perfect. Sea Eagles are oddly priced at 2.1. The less said about Manly, the better.

The Sydney Roosters are squaring off against the Brisbane Broncos. Roosters are expected to take this one home and are priced at a decent 2.41. Sydney boys versus the Brisbane blokes, sorry cobbers up north, but my bets are on the Sydney lads.

Not to forget, our blokes in red and blue, the mighty Newcastle Knights, are up against the Canberra Raiders. The Knights are set with a win p

In [ ]:
sf.send_emails(
    "footy-tipper-email-list", 
    f"Footy Tipper Predictions for {predictions['round_name'].unique()[0]}", 
    reg_reagan, 
    os.getenv('MY_EMAIL'), 
    os.getenv('EMAIL_PASSWORD'), 
    json_path
)